In [1]:
import json
import requests
from config import api_key

In [2]:
# Basic structure of API call:
# year.dev-category.dev-friendly-variable-name.

In [3]:
# Params
year = 2018 
state = 'FL'
predom_type = 2,3
# Grab the base url
url = f"http://api.data.gov/ed/collegescorecard/v1/schools.json?/school.degrees_awarded.predominant=2&fields=id,school.name/{year}/{state}/api_key={api_key}"
response = requests.get(url).json()

#specifying areas of studies
    1: Undergraduate Certificates or Diplomas
    2: Associate’s Degrees
    3: Bachelor’s Degrees
    4: Post-Baccalaureate Certificates
    5: Master’s Degrees
    6: Doctoral Degrees
    7: First Professional Degrees
    8: Graduate / Professional Certificates

In [5]:
# https://api.data.gov/ed/collegescorecard/v1/schools.json?school.degrees_awarded.predominant=2,3&fields=id,school.name,2013.student.size


In [14]:

# url_2 = f"https://api.data.gov/ed/collegescorecard/v1/schools.json?school.degrees_awarded.predominant={predom_type}.school.state={state}&fields=id,school.name,school.city,school.state,{year}.cost.attendance.academic_year,{year}.cost.avg_net_price.public,{year}.cost.tuition.in_state&api_key={api_key}"

# responses = requests.get(url_2).json()
# responses
url_3 = f"https://api.data.gov/ed/collegescorecard/v1/schools.json?school.degrees_awarded.predominant=2,3&fields=id,school.state={state}&fields=id,school.name,{year}.cost.attendance.academic_year&api_key={api_key}"
response_1 = requests.get(url_3).json()
response_1

{'metadata': {'total': 3203, 'page': 0, 'per_page': 20},
 'results': [{'school.name': 'Platt College-Miller-Motte Technical-Columbus',
   'id': 458441,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'Bryant & Stratton College-Hampton',
   'id': 459107,
   '2018.cost.attendance.academic_year': 22982},
  {'school.name': 'Fortis College-Salt Lake City',
   'id': 456454,
   '2018.cost.attendance.academic_year': 27717},
  {'school.name': 'Ambria College of Nursing',
   'id': 457527,
   '2018.cost.attendance.academic_year': None},
  {'school.name': 'Houston International College Cardiotech Ultrasound School',
   'id': 458034,
   '2018.cost.attendance.academic_year': 33657},
  {'school.name': 'Moreno Valley College',
   'id': 460394,
   '2018.cost.attendance.academic_year': 12395},
  {'school.name': 'Norco College',
   'id': 460464,
   '2018.cost.attendance.academic_year': 12031},
  {'school.name': 'Rasmussen College-Kansas',
   'id': 480657,
   '2018.cost.attendance.academ